In [1]:
import tabula
import pandas as pd
import glob
import numpy as np
import re

In [ ]:
# Specify the folder path containing the PDF files
pdf_folder_path = r"C:\Users\Learner_XZHCG221\Budget_Sheet\Statement"

# Get a list of PDF files in the folder
pdf_files = glob.glob(pdf_folder_path + "/*.pdf")

In [ ]:
# Initialize an empty list to store DataFrames for each PDF
dfs = []

# Iterate over each PDF file
for pdf_file in pdf_files:
    try:
         #Read tables from the PDF file

        tables = tabula.read_pdf(pdf_file, pages='all', multiple_tables=True)

        # Check if any tables were extracted
        if tables:
            # Extract the desired table from the list of tables
            # (You may need to adjust the index based on the structure of your statement)
            table = tables[0]

            # Append the table DataFrame to the list
            dfs.append(table)
    except Exception as e:
        print(f"Error occurred while processing {pdf_file}: {str(e)}")

In [4]:
# Check if any tables were extracted
if dfs:
    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv('combined_statements.csv', index=False)
else:
    print("No tables found in the PDF files.")

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('combined_statements.csv')
df

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df.dtypes

In [ ]:
import pandas as pd

# Combine columns into Transaction Details
df['Transaction Details'] = df['DEPOSITS AND ADDITIONS'].fillna('') + df['ATM & DEBIT CARD WITHDRAWALS'].fillna('') + df['ATM & DEBIT CARD WITHDRAWALS (continued)'].fillna('')

# Combine corresponding amounts into Transaction Amount
df['Transaction Amount'] = df['Unnamed: 0'].fillna('') + df['Unnamed: 1'].fillna('')

# Drop the unnecessary columns
df = df.drop(['DEPOSITS AND ADDITIONS', 'ATM & DEBIT CARD WITHDRAWALS', 'ATM & DEBIT CARD WITHDRAWALS (continued)', 'Unnamed: 0', 'Unnamed: 1'], axis=1)

# Print the updated DataFrame
print(df)




To combine two columns where the values from one column are appended to the values from the other column, you can use the pd.concat() function in Pandas. Here's an example:

In [ ]:
df['Transaction Amount'] = pd.concat([df['Unnamed: 0'], df['Unnamed: 1']], ignore_index= True)

df= df.drop(['Unnamed: 0', 'Unnamed: 1'], axis= 1)

print(df)

In [ ]:
df['Transaction Details']= pd.concat([df['DEPOSITS AND ADDITIONS'], df['ATM & DEBIT CARD WITHDRAWALS'], df['ATM & DEBIT CARD WITHDRAWALS (continued)']], ignore_index=True)

df= df.drop(['DEPOSITS AND ADDITIONS', 'ATM & DEBIT CARD WITHDRAWALS', 'ATM & DEBIT CARD WITHDRAWALS (continued)'], axis= 1)

df

In [ ]:
# Convert 'Transaction Amount' column to string type
df['Transaction Amount'] = df['Transaction Amount'].astype(str)

# Iterate through the 'Transaction Amount' column and delete rows with the string 'Amount'
for index, row in df.iterrows():
    if 'AMOUNT' in row['Transaction Amount']:
        df.drop(index, inplace=True)

# Reset the index of the DataFrame after deleting rows
df.reset_index(drop=True, inplace=True)

# Print the updated DataFrame
print(df)

In [ ]:
# Remove currency symbol ('$')
df['Transaction Amount'] = df['Transaction Amount'].str.replace('$', '')

# Remove ','
df['Transaction Amount'] = df['Transaction Amount'].str.replace(',', '')

# Convert empty strings to NaN
df['Transaction Amount'].replace('', float('nan'), inplace=True)

# Convert 'Transaction Amount' column to float type
df['Transaction Amount'] = df['Transaction Amount'].astype(float)

df['Transaction Amount'] = df['Transaction Amount'].apply(lambda x: '{:.2f}'.format(x))

# Print the updated DataFrame
df



In [ ]:
#df.to_csv('first.csv', index=False)

In [ ]:
# Filter the DataFrame to only keep columns with 'Card Purchase'
df = df[df['Transaction Details'].str.contains('Card Purchase', case=False, na=False)]

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

df


In [ ]:
# Function to extract the first date from the string
def extract_date(transaction):
    date = re.search(r'\d{2}/\d{2}', transaction)  # Assuming the date format is MM/DD
    if date:
        extracted_date = date.group()
        return f'{extracted_date}/2022'  # Add the year
    else:
        return None

# Apply the function to extract the first date and create a new 'Date' column
df['Date'] = df['Transaction Details'].apply(extract_date)

df


In [ ]:
df['Date'].unique()

In [ ]:
# Function to label transactions based on specific keywords
def label_transaction(transaction):
    keywords_groceries = ['Wholefds', "Trader Joe's", 'Foodtown', 'Walmart', 'Wal-MArt', 'King Kullen', 'King Kull', 'Stop & Shop',
                          'Food Lion']
    keywords_food_drink = [ 'Chipotle',"Dunns River", 'Starbucks', 'Coldstone', 'Chimichurri Chicken', 'LA Burdick Chocolate', 
                           'Wild Ginger', 'Life Cafe', 'Magic Bagels', 'Bww', '5Guys', 'Five Guys', 'DoorDash', 
                           "Henrica's", 'Chick-Fil-A', "Chili's", 'New Country Delicat', 'Succotash', 'Capital Grille', 
                           'Four Burritos Diner', 'Pdq', "Frank Pepe's Pizze", 'New Country Deli & Syosset', 'Grillfire',
                           'Sweet Peace Bake House']
    keywords_shopping= ['Target', 'Walgreens','cvs', 'Bloomys', 'Michaels', 'Staples']
    keywords_bills= ['Goldberg', 'Women For Women', 'Gulf Oil', 'E-Z', 'Endocare', 'Lexus', 'Northwell', 'Central Valley']
    keywords_misc = ['Bellerose Nail','Fandango', 'Transcript', 'Cinemark', '7-Eleven', 
                     'Source Serv', 'Safeway', 'Pre-Pt', 'Bp', 'Highs', 'Bodybuilding', 'Casper', 'Adinkra', 
                     'European Wax Center', 'Robbins Mart', 'NH J Garage', 'Ipic', 'Colpark', 'Apb', 'Topgolf', 
                     'Shell', 'Convenience', 'Autozone', 'Snaxx', 'Vitamin Shoppe', 'Apple.Com', 'Chandni Hair And Mak', 'New Jersey', 
                     'Apple Store', 'Hair World', 'Hermes', 'Elkridge Eigen', 'Tst* Thb', 'Fh* 5 Wits' ]
    
    for keyword in keywords_groceries:
        if keyword.lower() in transaction.lower():
            return 'Groceries'
    for keyword in keywords_food_drink:
        if keyword.lower() in transaction.lower():
            return 'Food & Drink'
    for keyword in keywords_shopping:
        if keyword.lower() in transaction.lower():
            return 'Shopping'
    for keyword in keywords_bills:
        if keyword.lower() in transaction.lower():
            return 'Bills'
    for keyword in keywords_misc:
        if keyword.lower() in transaction.lower():
            return 'Misc'
    return None

# Apply the function to label transactions and create a new 'Transaction Type' column
df['Transaction Type'] = df['Transaction Details'].apply(label_transaction)

# Print the updated DataFrame
df


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.to_csv('cleaned_statments.csv', index=False)